# Chrome Built-in AI Demo in JupyterLab and JupyterLite Python Notebook

This notebook demonstrates the Python wrapper for Chrome's built-in AI Prompt API.

https://developer.chrome.com/docs/ai/prompt-api

**Requirements:**
- Chrome browser with Prompt API enabled
- Running in JupyterLab, Jupyter Notebook, or JupyterLite (statically served, all execution locally in browser)

## For developers: Using the Prompt API
* Open Chrome and go to `chrome://flags`.
* Enable the `#prompt-api-for-gemini-nano` flag.
* Enable the `#optimization-guide-on-device-model` flag. If you see a "BypassPerfRequirement" option, select it.
* Restart Chrome.

In [ ]:
%pip install -q anywidget ipywidgets wiki3_ai

In [ ]:
%gui asyncio

In [ ]:
# Import the library
from wiki3_ai import LanguageModel, Availability, LanguageModelWidget

In [ ]:
lm = LanguageModel()
lm.widget

In [ ]:
session_id = await lm.create()
session_id

## Check Availability

First, let's check if the Prompt API is available in your browser.

In [9]:
# Check if the API is available
await lm.availability()

Sending request ID:  bf5ba2a5-4470-46dd-9bdf-0f79caf38b8a
Sent request:  {'id': 'bf5ba2a5-4470-46dd-9bdf-0f79caf38b8a', 'method': 'availability', 'params': {'options': {}}}


<Availability.AVAILABLE: 'available'>

## Simple Prompt

Create a session and send a simple prompt.

In [10]:
# Send a prompt
result = await lm.prompt("Write a haiku about Python programming.")
result

Sending request ID:  699491c0-62a4-4098-9411-b4b90b0784b5
Sent request:  {'id': '699491c0-62a4-4098-9411-b4b90b0784b5', 'method': 'prompt', 'params': {'sessionId': 'bd47ff02-a718-45dc-96d0-db868e947ed6', 'input': 'Write a haiku about Python programming.', 'options': {}}}


'Code flows, clean and bright,\nEasy to learn, so versatile,\nScripts bloom, tasks are done. \n\n\n\n'

## System Prompt

Use a system prompt to set the context for the conversation.

In [11]:
# Create a session with a system prompt
assistant_session = await LanguageModel.create(
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Python programming assistant who gives concise answers.",
            }
        ]
    }
)

# Ask a question
result = await assistant_session.prompt("How do I read a CSV file in Python?")
print(result)

AttributeError: 'dict' object has no attribute 'widget'

## Streaming Response

Stream the response as it's generated.

In [13]:
print("Response: ", end="")
async for chunk in lm.prompt_streaming("Tell me a short joke about computers."):
    print(chunk, end="", flush=True)
print()

Response: Sending request ID:  431eca0d-29bc-44c9-8014-d6c99abf4647
Sent request:  {'id': '431eca0d-29bc-44c9-8014-d6c99abf4647', 'method': 'promptStreaming', 'params': {'sessionId': 'bd47ff02-a718-45dc-96d0-db868e947ed6', 'requestId': '310051d8-0304-466e-9243-b4b1e9041680', 'input': 'Tell me a short joke about computers.', 'options': {}}}



## Multi-turn Conversation

Have a conversation with context maintained across prompts.

In [ ]:
# First message
result1 = await assistant_session.prompt("What is a list comprehension?")
print("Assistant:", result1)
print()

# Follow-up message (the assistant remembers the context)
result2 = await assistant_session.prompt("Can you show me an example?")
print("Assistant:", result2)

## Check Token Usage

Monitor how many tokens you've used.

In [ ]:
print(f"Current usage: {assistant_session.input_usage}/{assistant_session.input_quota} tokens")

# Measure how many tokens a prompt would use
usage = await assistant_session.measure_input_usage("What is machine learning?")
print(f"\nThis prompt would use approximately {usage} tokens")

## Get Model Parameters

Let's see what parameters the model supports.

In [ ]:
params = await lm.params()
params

In [ ]:
if params:
    print(f"Default temperature: {params.default_temperature}")
    print(f"Max temperature: {params.max_temperature}")
    print(f"Default top-K: {params.default_top_k}")
    print(f"Max top-K: {params.max_top_k}")
else:
    print("Model parameters not available")

## Structured Output

Use JSON schema to get structured responses.

In [ ]:
import json

# Define a JSON schema
schema = {
    "type": "object",
    "required": ["sentiment", "score"],
    "properties": {
        "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
        "score": {"type": "number", "minimum": 0, "maximum": 1},
    },
}

# Get structured response
result = await session.prompt(
    "Analyze this review: The product exceeded all my expectations! Absolutely amazing!",
    {"responseConstraint": schema},
)

# Parse the JSON response
data = json.loads(result)
print(f"Sentiment: {data['sentiment']}")
print(f"Score: {data['score']}")

## Session Cloning

Clone a session to create different conversation branches.

In [ ]:
# Start a story
story_session = await LanguageModel.create(
    {"initialPrompts": [{"role": "system", "content": "You are a creative storyteller."}]}
)

await story_session.prompt("Once upon a time, there was a dragon.")

# Create two different story branches
branch1 = await story_session.clone()
branch2 = await story_session.clone()

result1 = await branch1.prompt("The dragon was friendly and helpful.")
print("Branch 1:", result1)
print()

result2 = await branch2.prompt("The dragon was fierce and terrifying.")
print("Branch 2:", result2)

## Cleanup

Destroy sessions when you're done to free up resources.

In [ ]:
await session.destroy()
await assistant_session.destroy()
await story_session.destroy()
await branch1.destroy()
await branch2.destroy()

print("✅ All sessions destroyed")